In [1]:
import xml.etree.ElementTree as ET
import os
import cPickle
import numpy as np

In [2]:
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects

In [3]:
def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [4]:
def read_classnames(classname):
    return [line.split(' ')[0][:-1] for line in open(r'E:\dataset\Animal\imagenet\\' + classname + '_wnids.txt').readlines()]

In [5]:
def extract_wnid(imagename):
    return imagename[:imagename.rfind('_')]

In [21]:
def extract_image_id(full_id):
    ind = full_id.rindex('\\')
    return full_id[ind+1:-1]
    

In [35]:
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             ovthresh=0.5,
             use_07_metric=False):
    """rec, prec, ap = voc_eval(detpath,
                                annopath,
                                imagesetfile,
                                classname,
                                [ovthresh],
                                [use_07_metric])
    Top level function that does the PASCAL VOC evaluation.
    detpath: Path to detections
        detpath.format(classname) should produce the detection results file.
    annopath: Path to annotations
        annopath.format(wnid,imagename) should be the xml annotations file.
    imagesetfile: Text file containing the list of images, one image per line.
    classname: Category name (duh)
    cachedir: Directory for caching the annotations
    [ovthresh]: Overlap threshold (default = 0.5)
    [use_07_metric]: Whether to use VOC07's 11 point AP computation
        (default False)
    """
    # assumes detections are in detpath.format(classname)
    # assumes annotations are in annopath.format(imagename)
    # assumes imagesetfile is a text file with each line an image name
    # cachedir caches the annotations in a pickle file

    #read classnames in  ImagenetFormat
    imagenet_classnames = read_classnames(classname)
    
    # first load gt

    # read list of images
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]

    
    # load annots
    recs = {}
    for i, imagename in enumerate(imagenames):
        recs[imagename] = parse_rec(annopath.format(extract_wnid(imagename),imagename))
        #if i % 100 == 0:
            #print 'Reading annotation for {:d}/{:d}'.format(i + 1, len(imagenames))
    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] in imagenet_classnames]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    # read dets
    detfile = detpath.format(classname)
    with open(detfile, 'r') as f:
        lines = f.readlines()

    splitlines = [x.strip().split(' ') for x in lines]
    image_ids = [extract_image_id(repr(x[0])) for x in splitlines]
    confidence = np.array([float(x[1]) for x in splitlines])
    BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    for d in range(nd):
        R = class_recs[image_ids[d]]
        bb = BB[d, :].astype(float)
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:
            # compute overlaps
            # intersection
            ixmin = np.maximum(BBGT[:, 0], bb[0])
            iymin = np.maximum(BBGT[:, 1], bb[1])
            ixmax = np.minimum(BBGT[:, 2], bb[2])
            iymax = np.minimum(BBGT[:, 3], bb[3])
            iw = np.maximum(ixmax - ixmin + 1., 0.)
            ih = np.maximum(iymax - iymin + 1., 0.)
            inters = iw * ih

            # union
            uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
                   (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                   (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

            overlaps = inters / uni
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)

        if ovmax > ovthresh:
            if not R['difficult'][jmax]:
                if not R['det'][jmax]:
                    tp[d] = 1.
                    R['det'][jmax] = 1
                else:
                    fp[d] = 1.
        else:
            fp[d] = 1.

    # compute precision recall
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)

    return rec, prec, ap

In [65]:

old_mAPs = old_mAPs + mAPs
print linspace(0,10,1)

[ 0.]


In [70]:
lines = [ line for line in open('C:\\cfnet\data\cfnet-validation\validation/tc_Kite_ce3/groundtruth.txt') ] 


IOError: [Errno 22] invalid mode ('r') or filename: 'C:\\cfnet\\data\\cfnet-validation\x0balidation/tc_Kite_ce3/groundtruth.txt'

In [71]:
import matplotlib.pyplot  as plt
from pylab import *
import numpy as np
  

classnames = ['person','animal','bird']
results = {}
mAPs = []
for iteration in range(45,108):
    print 'iter: {}'.format(iteration)
    for classname in classnames:
        results[classname] = voc_eval(r'C:\darknet\build\darknet\x64\results\comp4_det_test_{0}-animal2-' + str(iteration) + '000.txt',
                r'E:\dataset\Animal\imagenet\synsets\{0}\annotations\{1}.xml',
                r'E:\dataset\Animal\imagenet\imagesetfile.txt',
                classname)
    mAP = 0
    fig= plt.figure()
    for i,classname in enumerate(classnames):
        rec,prec,ap = results[classname]
        print rec.shape,prec.shape
        mAP+=ap
        ax = fig.add_subplot(221+i)  
        plt.title(classname)
        plt.plot(rec,np.linspace(0,1,len(rec))/len(rec),label='recall')
        plt.plot(prec,np.linspace(0,1,len(prec))/len(prec),label='precision')
        plt.legend()
        plt.grid()
        fig.subplots_adjust(hspace=.5)
        ax.set_yticks(np.linspace(0,1,11))
        mAP = mAP/len(classnames)
    mAPs.append(mAP)

    ax = fig.add_subplot(224)
    plt.title('mAP')
    ax.bar(1,mAP,0.05)

    ax.set_yticks(np.linspace(0,1,11))
    ax.set_yticks(np.linspace(0,1,11))
    savefig('animal2_evaluation_last{}.jpg'.format(iteration),dpi=200)
    #plt.show()


iter: 45
(10739L,) (10739L,)
(37107L,) (37107L,)
(41631L,) (41631L,)
iter: 46
(12344L,) (12344L,)
(38926L,) (38926L,)
(43175L,) (43175L,)
iter: 47
(12047L,) (12047L,)
(38638L,) (38638L,)
(42483L,) (42483L,)
iter: 48
(11980L,) (11980L,)
(38560L,) (38560L,)
(42888L,) (42888L,)
iter: 49
(12116L,) (12116L,)
(40315L,) (40315L,)
(43950L,) (43950L,)
iter: 50
(12344L,) (12344L,)
(39548L,) (39548L,)
(43170L,) (43170L,)
iter: 51
(11993L,) (11993L,)
(39021L,) (39021L,)
(44210L,) (44210L,)
iter: 52
(11860L,) (11860L,)
(38915L,) (38915L,)
(42532L,) (42532L,)
iter: 53
(11687L,) (11687L,)
(38345L,) (38345L,)
(41721L,) (41721L,)
iter: 54
(11788L,) (11788L,)
(38558L,) (38558L,)
(43066L,) (43066L,)
iter: 55
(11400L,) (11400L,)
(38666L,) (38666L,)
(42591L,) (42591L,)
iter: 56
(11650L,) (11650L,)
(38793L,) (38793L,)
(42273L,) (42273L,)
iter: 57
(12168L,) (12168L,)
(39170L,) (39170L,)
(42651L,) (42651L,)
iter: 58
(11975L,) (11975L,)
(38273L,) (38273L,)
(42828L,) (42828L,)
iter: 59
(12023L,) (12023L,)
(3931

In [56]:
fig= plt.figure()
mAPs = old_mAPs + mAPs

plt.plot(mAPs)
plt.ylabel('mean Average Precision')
plt.xlabel('number of iterations in 1000s')
plt.yticks(np.linspace(0,0.5,11))
plt.grid()
savefig(r'E:\Work\Animal Detection\3700-images\animal2_mAPs.png',dpi=200)
#plt.show()

In [60]:
print len(old_mAPs)

44
